In [83]:
import pandas as pd
import numpy as np
from common_utilities import global_path

In [84]:
# read the csv file
df_TradeHistory = pd.read_csv(global_path.tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")

# replace scrip code to compnay name
df_Symbol = pd.read_csv(global_path.symbol_silver_file_path)
df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)

# Merge df_TradeHistory with df_Symbol on the matching columns
df_TradeHistory = df_TradeHistory.merge(
    df_Symbol[["scrip_code", "symbol"]],
    left_on="scrip_code",
    right_on="scrip_code",
    how="left",
)

# Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(df_TradeHistory["stock_name"])

In [85]:
class PnlSnapshot:
    def __init__(self):
        self.net_position = 0
        self.avg_price = 0

    def trade(self, side, traded_price, traded_quantity):
        # buy: positive position, sell: negative position
        quantity_with_direction = traded_quantity if side == "BUY" else (-1) * traded_quantity
        is_still_open = (self.net_position * quantity_with_direction) >= 0
        realized_pnl = 0
        # realized pnl
        if not is_still_open:
            # Remember to keep the sign as the net position
            realized_pnl = (
                (traded_price - self.avg_price)
                * min(abs(quantity_with_direction), abs(self.net_position))
                * (abs(self.net_position) / self.net_position)
            )
        # avg open price
        if is_still_open:
            self.avg_price = ((self.avg_price * self.net_position) + (traded_price * quantity_with_direction)) / (
                self.net_position + quantity_with_direction
            )
        else:
            # Check if it is close-and-open
            if traded_quantity > abs(self.net_position):
                self.avg_price = traded_price
        # net position
        self.net_position += quantity_with_direction
        return {"net_position": self.net_position, "avg_price": self.avg_price, "realized_pnl": realized_pnl}

In [86]:
portfolio = {}

df = df_TradeHistory.sort_values(by="datetime")
data = []

for _, row in df.iterrows():
    row = row.to_dict()
    if row["stock_name"] not in portfolio:
        portfolio[row["stock_name"]] = PnlSnapshot()
    row.update(portfolio[row["stock_name"]].trade(row["side"], row["price"], row["quantity"]))
    data.append(row)

pd.DataFrame(data)[
    ["datetime", "exchange", "segment", "stock_name", "side", "quantity", "price", "amount", "net_position", "avg_price", "realized_pnl"]
]

,datetime,exchange,segment,stock_name,side,quantity,price,amount,net_position,avg_price,realized_pnl
0,2020-04-21 14:41:30,NSE,EQ,TATAMOTORS,BUY,14.0,75.10,1051.40,14.0,75.100000,0.000
1,2020-05-04 14:33:45,NSE,EQ,TATAMOTORS,BUY,5.0,83.80,419.00,19.0,77.389474,0.000
2,2020-05-05 11:31:20,NSE,EQ,BHAGERIA,BUY,10.0,116.55,1165.50,10.0,116.550000,0.000
3,2020-05-19 14:44:06,NSE,EQ,BHAGERIA,BUY,5.0,100.00,500.00,15.0,111.033333,0.000
4,2020-06-15 10:57:42,NSE,EQ,BHAGERIA,SELL,15.0,113.00,1695.00,0.0,111.033333,29.500
...,...,...,...,...,...,...,...,...,...,...,...
170,2024-07-23 12:22:03,FON,FO,NIFTY-CE-24500-25Jul2024,SELL,25.0,108.95,2723.75,0.0,205.325000,-2409.375
171,2024-07-26 11:34:24,FON,FO,NIFTY-PE-24650-01Aug2024,BUY,25.0,138.00,3450.00,25.0,138.000000,0.000
172,2024-07-26 12:20:50,FON,FO,NIFTY-CE-24600-01Aug2024,BUY,25.0,190.70,4767.50,25.0,190.700000,0.000
173,2024-07-26 12:33:43,FON,FO,NIFTY-PE-24650-01Aug2024,SELL,25.0,106.15,2653.75,0.0,138.000000,-796.250
